**Project Summary:**

Telco customers are leaving for newer companies. Especially phone customers. Find a driver of the churn. A copy of the report will go to executives. Include enough documentation in your final notebook that someone could work through it without you.

**Goals:**
- Find drivers for customer churn at Telco
- Construct a ML classification model that accurately predicts customer churn. 

**Process/Pipeline:**  https://trello.com/b/vOXbVcbl





In [1]:
import pandas as pd
import acquire
import env
from acquire import get_telco_data

In [5]:
#Acquire the data from the telco_churn database on the Codeup data science database server

df = get_telco_data()
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,internet_service_type_id,internet_service_type,contract_type_id,contract_type,payment_type_id,payment_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,2,59.9,542.4,No,1,DSL,1,Month-to-month,2,Mailed check
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,4,69.4,571.45,No,1,DSL,1,Month-to-month,4,Credit card (automatic)
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,1,48.2,340.35,No,1,DSL,1,Month-to-month,1,Electronic check
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,1,25.1,25.1,Yes,1,DSL,1,Month-to-month,1,Electronic check
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,3,30.5,30.5,Yes,1,DSL,1,Month-to-month,3,Bank transfer (automatic)


In [7]:
df.shape

(7043, 27)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               7043 non-null   object 
 1   gender                    7043 non-null   object 
 2   senior_citizen            7043 non-null   int64  
 3   partner                   7043 non-null   object 
 4   dependents                7043 non-null   object 
 5   tenure                    7043 non-null   int64  
 6   phone_service             7043 non-null   object 
 7   multiple_lines            7043 non-null   object 
 8   internet_service_type_id  7043 non-null   int64  
 9   online_security           7043 non-null   object 
 10  online_backup             7043 non-null   object 
 11  device_protection         7043 non-null   object 
 12  tech_support              7043 non-null   object 
 13  streaming_tv              7043 non-null   object 
 14  streamin

In [ ]:
Data dictionary: 
customer_id - object
gender - object
senior_citizen int64
partner object
dependents - object
tenure - int64
phone_service - object
multiple_line - object
internet_service_type_id - int 64
online_security - object
online_backup - object
device_protection - object
tech_support - object
streaming_tv - object
streaming_movies - object
contract_type_id - int64
paperless_billing - object
payment_type_id - int64
monthly_charges - float64
total_charges - object
churn - object
internet_service_type_id int64
internet_service_type - object
contract_type_id - int64
contract_type 0 object
payment_type_id - int64
payment_type - object

    

In [ ]:
I want to:

- encode:
    - partner
    - dependents
    - phone_service
- make dummies:
    - gender
    - multiple_lines
    - online_security
    - online_backup
    - device_protection
    - tech_support
    - streaming_tv
    
- rename col:
    - tenure:tenure_months
    - internet_service_type_id:int_serv_type

In [ ]:
Notes for prep:  
df.total_charges = pd.to_numeric(df.total_charges, errors='coerce').astype('float64')

df.total_charges = df.total_charges.fillna(value=0)